In [2]:
# import libraris
import pandas as pd
import json
import requests
import re
from bs4 import BeautifulSoup

def Program_details_Mapper():

    # Opening JSON file
    with open('loyalist_college_data/loyalist_colleges.json', 'r') as openfile:

        # Reading from json file
        json_object = json.load(openfile)

    # Define Functios 
    def Program_name():
        pattern = '\A.*'
        return [''.join(re.findall(pattern, string)) for string in json_object]

    def Credential():
        pattern = '-\sCredential\n(.*)'
        return [''.join(re.findall(pattern,string)) for string in json_object]

    def Start_Date():
        pattern = '-\sStart\sDate\n(.*)'
        return [''.join(re.findall(pattern,string)) for string in json_object]

    def Location():
        pattern = '-\sLocation\n(.*\sCampus).*'
        return [''.join(re.findall(pattern,string)) for string in json_object]

    def Discriptions():
        pattern = "^[F|f]ind\s[y|Y]our\s[c|C]areer(.*)Is\sit\sfor\syou/?"
        return [''.join(re.findall(pattern,string,re.MULTILINE | re.DOTALL)).replace('\n',' ') for string in json_object]

    def Is_it_for_you():
        pattern = "^Is\sit\sfor\syou\?(.*)Experiential\slearning"
        return [''.join(re.findall(pattern,string,re.MULTILINE | re.DOTALL)).replace('\n',' ') for string in json_object]

    def Duration():
        Duration = []
        pattern = "^.*\sYear\s-\sSemester\s.*$"
        Year_Semester = [''.join(re.findall(pattern,string,re.MULTILINE )).replace('\n',' ') for string in json_object]
        for year in Year_Semester:
            if re.findall("Fourth Year", year) :
                Duration.append('4 Year')
            elif re.findall("Third Year", year) :
                Duration.append('3 Year')
            elif re.findall("Second Year", year) :
                Duration.append('2 Year')
            elif re.findall("First Year", year) :
                Duration.append('1 Year')
            else :
                Duration.append(' ')
        return Duration

    def Semester():
        Semester = []
        pattern = "^.*\sYear\s-\sSemester\s.*$"
        Year_Semester = [''.join(re.findall(pattern,string,re.MULTILINE )).replace('\n',' ') for string in json_object]
        for year in Year_Semester:
            if re.findall("Fourth Year", year) :
                Semester.append('8 Semester')
            elif re.findall("Third Year", year) :
                Semester.append('6 Semester')
            elif re.findall("Second Year", year) :
                Semester.append('4 Semester')
            elif re.findall("First Year", year) :
                Semester.append('2 Semester')
            else :
                Semester.append(' ')
        return Semester

    def Tuition_Fee():
        pattern = "Approximate\scosts.*\n.*:\s(.*)\n.*\n.*"
        return [''.join(re.findall(pattern,string)).replace('\n',' ') for string in json_object]

    def Ancillary_Fees():
        pattern = "Approximate\scosts.*\n.*\n.*(\$.*)\n.*\n.*"
        return [''.join(re.findall(pattern,string)).replace('\n',' ') for string in json_object]

    def Mode():
        pattern = "Approximate\scosts.*\n.*\n-\s(Full-Time|Online)\s.*\n.*\n.*"
        return [''.join(re.findall(pattern,string)).replace('\n',' ') for string in json_object]

    def Admission_requirements():
        pattern = "with\sa\sstudent\sloan\.(.*)\[Click\shere\]"
        return [''.join(re.findall(pattern,string,re.MULTILINE | re.DOTALL)).replace('\n',' ') for string in json_object]

    def Link_of_the_program():   
        base_url = 'https://www.loyalistcollege.com/programs-and-courses/full-time-programs/'
        page_source = requests.get(base_url)
        soup = BeautifulSoup(page_source.content, 'html.parser',from_encoding="utf-8")
        raw_urls = []
        for link in soup.find_all('a'):        
            raw_urls.append(link.get('href'))
        raw_urls = list(raw_urls)
        sub_urls = []
        base = 'https://www.loyalistcollege.com'
        pattern ='^\/programs-and-courses\/full-time-programs\/.+'
        for link in raw_urls:
            if link != None:
                if re.search(pattern, link):
                    sub_urls.append(base + link)
        sub_urls = set(sub_urls)
        sub_urls = list(sub_urls)
        sub_urls.sort()
        return sub_urls

    # create mapping with dictinary
    data_dict = {
        'Institution Name'              :'Loyalist College Canada',
        'Program Name'                  :Program_name(),
        'Mode'                          :Mode(),
        'Duration of program'           :Duration(),
        'No of Semester'                :Semester(),
        'Currency used in the country'  :'CAD$',
        'Level of Education(Credential)':Credential(),
        'Start Date'                    :Start_Date(),
        'Location'                      :Location(),
        'Discriptions'                  :Discriptions(),
        'Is it for you ?'               :Is_it_for_you(),
        'Tuition Fee'                   :Tuition_Fee(),
        'Ancillary Fees'                :Ancillary_Fees(),
        'Admission requirements'        :Admission_requirements(),
        'Link of the program'           :Link_of_the_program()
                 }
    # convert into dataframe
    df = pd.DataFrame(data=data_dict)

    #convert into excel
    df.to_excel("loyalist_college_data/LOYALiST_COLLEGE_Details.xlsx", index=False)
    

Program_details_Mapper()